# QoL Index Dashboard

`Nika Karimi Jashni, Tie Ma, Yunhan Liu` <br>

Last Updated: June 6, 2024

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output

# Sample data: Replace this with your actual data
data = {
    'City': ['Montreal', 'Vancouver', 'Winnipeg', 'Kingston', 'Kitchener', 'Quebec City', 'Ottawa', 'Toronto', 'Edmonton', 'Calgary', 'Hamilton'],
    'Latitude': [45.5, 49.2, 49.9, 44.2, 43.4, 46.8, 45.4, 43.7, 53.5, 51.0, 43.2],
    'Longitude': [-73.6, -123.1, -97.1, -76.5, -80.5, -71.2, -75.7, -79.4, -113.5, -114.1, -79.9],
    'Environment': [1, 2, 3, 4, 5, 2, 3, 4, 5, 3, 4],
    'Society': [5, 4, 3, 2, 1, 3, 4, 5, 2, 1, 5],
    'Access': [2, 3, 4, 1, 5, 4, 1, 5, 3, 2, 1],
    'Prospect': [3, 1, 2, 5, 4, 5, 1, 3, 4, 2, 3],
    'Affordability': [4, 5, 1, 3, 2, 1, 3, 2, 5, 4, 2]
}
df = pd.DataFrame(data)

# Initialize the Dash app
app = dash.Dash(__name__)

# External stylesheets for better visuals
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# Layout of the dashboard
app.layout = html.Div([
    html.Div([
        html.Div([
            html.H2("City Value", style={'textAlign': 'center'}),
            dcc.Dropdown(
                id='city-dropdown',
                options=[{'label': city, 'value': city} for city in df['City']],
                value='Montreal'
            ),
            html.Div(id='city-value'),
            html.H3("City Rank", style={'textAlign': 'center'}),
            html.Div(id='city-rank')
        ], style={'width': '20%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '20px', 'backgroundColor': '#f0f0f0'}),
        
        html.Div([
            html.H1("Map", style={'textAlign': 'center'}),
            html.P("Highlight selected cities and Canada itself", style={'textAlign': 'center'}),
            dcc.Graph(id='map', style={'height': '600px'})
        ], style={'width': '60%', 'display': 'inline-block', 'padding': '20px'}),
        
        html.Div([
            html.H3("Scores", style={'textAlign': 'center'}),
            html.Div([
                html.P("Environment"),
                dcc.Slider(
                    id='slider-1',
                    min=0,
                    max=5,
                    step=1,
                    value=1,
                    marks={i: str(i) for i in range(6)}
                )
            ], style={'padding': '10px'}),
            html.Div([
                html.P("Society"),
                dcc.Slider(
                    id='slider-2',
                    min=0,
                    max=5,
                    step=1,
                    value=1,
                    marks={i: str(i) for i in range(6)}
                )
            ], style={'padding': '10px'}),
            html.Div([
                html.P("Access"),
                dcc.Slider(
                    id='slider-3',
                    min=0,
                    max=5,
                    step=1,
                    value=1,
                    marks={i: str(i) for i in range(6)}
                )
            ], style={'padding': '10px'}),
            html.Div([
                html.P("Prospects"),
                dcc.Slider(
                    id='slider-4',
                    min=0,
                    max=5,
                    step=1,
                    value=1,
                    marks={i: str(i) for i in range(6)}
                )
            ], style={'padding': '10px'}),
            html.Div([
                html.P("Affordability"),
                dcc.Slider(
                    id='slider-5',
                    min=0,
                    max=5,
                    step=1,
                    value=1,
                    marks={i: str(i) for i in range(6)}
                )
            ], style={'padding': '10px'})
        ], style={'width': '20%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '20px', 'backgroundColor': '#f0f0f0'})
    ], style={'display': 'flex', 'flexDirection': 'row', 'width': '100%'}),
    
    html.Div([
        html.H3("City Scores Table", style={'textAlign': 'center'}),
        html.Table(id='city-scores-table', style={'width': '100%', 'textAlign': 'center'})
    ], style={'width': '100%', 'display': 'inline-block', 'padding': '20px'})
], style={'display': 'flex', 'flexDirection': 'column'})


@app.callback(
    Output('map', 'figure'),
    [Input('slider-1', 'value'),
     Input('slider-2', 'value'),
     Input('slider-3', 'value'),
     Input('slider-4', 'value'),
     Input('slider-5', 'value')]
)
def update_map(weight1, weight2, weight3, weight4, weight5):
    # Calculate weighted average
    df['WeightedAvg'] = (weight1 * df['Environment'] + 
                         weight2 * df['Society'] + 
                         weight3 * df['Access'] + 
                         weight4 * df['Prospect'] + 
                         weight5 * df['Affordability'])

    # Create the map
    fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", size="WeightedAvg", 
                            hover_name="City", 
                            mapbox_style="carto-positron", 
                            size_max=15, zoom=3)
    
    return fig

@app.callback(
    Output('city-value', 'children'),
    [Input('city-dropdown', 'value')]
)
def update_city_value(selected_city):
    city_data = df[df['City'] == selected_city]
    return html.Div([
        html.H4(f"Values for {selected_city}"),
        html.P(f"Environment: {city_data['Environment'].values[0]}"),
        html.P(f"Society: {city_data['Society'].values[0]}"),
        html.P(f"Access: {city_data['Access'].values[0]}"),
        html.P(f"Prospect: {city_data['Prospect'].values[0]}"),
        html.P(f"Affordability: {city_data['Affordability'].values[0]}")
    ])

@app.callback(
    Output('city-rank', 'children'),
    [Input('slider-1', 'value'),
     Input('slider-2', 'value'),
     Input('slider-3', 'value'),
     Input('slider-4', 'value'),
     Input('slider-5', 'value')]
)
def update_city_rank(weight1, weight2, weight3, weight4, weight5):
    # Calculate weighted average
    df['WeightedAvg'] = (weight1 * df['Environment'] + 
                         weight2 * df['Society'] + 
                         weight3 * df['Access'] + 
                         weight4 * df['Prospect'] + 
                         weight5 * df['Affordability'])
    
    ranked_cities = df.sort_values(by='WeightedAvg', ascending=False)['City']
    return html.Ul([html.Li(city) for city in ranked_cities])

@app.callback(
    Output('city-scores-table', 'children'),
    [Input('slider-1', 'value'),
     Input('slider-2', 'value'),
     Input('slider-3', 'value'),
     Input('slider-4', 'value'),
     Input('slider-5', 'value')]
)
def update_city_scores_table(weight1, weight2, weight3, weight4, weight5):
    # Calculate weighted average
    df['WeightedAvg'] = (weight1 * df['Environment'] + 
                         weight2 * df['Society'] + 
                         weight3 * df['Access'] + 
                         weight4 * df['Prospect'] + 
                         weight5 * df['Affordability'])

    # Sort cities by weighted average
    sorted_df = df.sort_values(by='WeightedAvg', ascending=False)

    # Create table header
    header = html.Tr([
        html.Th("City"),
        html.Th("Environment"),
        html.Th("Society"),
        html.Th("Access"),
        html.Th("Prospect"),
        html.Th("Affordability"),
        html.Th("Weighted Avg")
    ])

    # Create table rows
    rows = [html.Tr([
        html.Td(city),
        html.Td(sorted_df.loc[sorted_df['City'] == city, 'Environment'].values[0]),
        html.Td(sorted_df.loc[sorted_df['City'] == city, 'Society'].values[0]),
        html.Td(sorted_df.loc[sorted_df['City'] == city, 'Access'].values[0]),
        html.Td(sorted_df.loc[sorted_df['City'] == city, 'Prospect'].values[0]),
        html.Td(sorted_df.loc[sorted_df['City'] == city, 'Affordability'].values[0]),
        html.Td(sorted_df.loc[sorted_df['City'] == city, 'WeightedAvg'].values[0])
    ]) for city in sorted_df['City']]

    # Combine header and rows
    table = [header] + rows

    return table

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)  # Change port number if 8050 is already in use


In [5]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output, State

# Sample data: Replace this with your actual data
data = {
    'City': ['Montreal', 'Vancouver', 'Winnipeg', 'Kingston', 'Kitchener', 'Quebec City', 'Ottawa', 'Toronto', 'Edmonton', 'Calgary', 'Hamilton'],
    'Latitude': [45.5, 49.2, 49.9, 44.2, 43.4, 46.8, 45.4, 43.7, 53.5, 51.0, 43.2],
    'Longitude': [-73.6, -123.1, -97.1, -76.5, -80.5, -71.2, -75.7, -79.4, -113.5, -114.1, -79.9],
    'Environment': [1, 2, 3, 4, 5, 2, 3, 4, 5, 3, 4],
    'Society': [5, 4, 3, 2, 1, 3, 4, 5, 2, 1, 5],
    'Access': [2, 3, 4, 1, 5, 4, 1, 5, 3, 2, 1],
    'Prospect': [3, 1, 2, 5, 4, 5, 1, 3, 4, 2, 3],
    'Affordability': [4, 5, 1, 3, 2, 1, 3, 2, 5, 4, 2]
}
df = pd.DataFrame(data)

# Initialize the Dash app
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.config.suppress_callback_exceptions = True

# Define CSS styles for sliders
slider_style = {
    'margin': '10px 0'
}

app.layout = html.Div([
    html.Button("Switch to French", id='language-switch', n_clicks=0),
    html.Div([
        html.Div([
            html.H2("City Value", id='city-value-title', style={'textAlign': 'center', 'color': '#333333'}),
            dcc.Dropdown(
                id='city-dropdown',
                options=[{'label': city, 'value': city} for city in df['City']],
                value='Montreal',
                style={'color': '#333333'}
            ),
            html.Div(id='city-value', style={'padding': '10px'}),
            html.H3("City Rank", id='city-rank-title', style={'textAlign': 'center', 'color': '#333333'}),
            html.Div(id='city-rank', style={'padding': '10px'})
        ], style={'width': '20%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '20px', 'backgroundColor': '#f7f7f7', 'borderRight': '1px solid #ccc'}),
        
        html.Div([
            html.H1("Map", id='map-title', style={'textAlign': 'center', 'color': '#333333'}),
            html.P("Highlight selected cities", id='map-subtitle', style={'textAlign': 'center', 'color': '#333333'}),
            dcc.Graph(id='map', style={'height': '700px', 'border': '1px solid #ccc', 'borderRadius': '5px'})
        ], style={'width': '60%', 'display': 'inline-block', 'padding': '20px', 'backgroundColor': '#ffffff'}),
        
        html.Div([
            html.H3("Scores", id='scores-title', style={'textAlign': 'center', 'color': '#333333'}),
            html.Div([
                html.P("Environment", id='environment-label', style={'color': '#333333'}),
                dcc.Slider(
                    id='slider-1',
                    min=0,
                    max=5,
                    step=1,
                    value=1,
                    marks={i: str(i) for i in range(6)},
                    className='blue-slider'
                )
            ], style=slider_style),
            html.Div([
                html.P("Society", id='society-label', style={'color': '#333333'}),
                dcc.Slider(
                    id='slider-2',
                    min=0,
                    max=5,
                    step=1,
                    value=1,
                    marks={i: str(i) for i in range(6)},
                    className='blue-slider'
                )
            ], style=slider_style),
            html.Div([
                html.P("Access", id='access-label', style={'color': '#333333'}),
                dcc.Slider(
                    id='slider-3',
                    min=0,
                    max=5,
                    step=1,
                    value=1,
                    marks={i: str(i) for i in range(6)},
                    className='blue-slider'
                )
            ], style=slider_style),
            html.Div([
                html.P("Prospects", id='prospects-label', style={'color': '#333333'}),
                dcc.Slider(
                    id='slider-4',
                    min=0,
                    max=5,
                    step=1,
                    value=1,
                    marks={i: str(i) for i in range(6)},
                    className='blue-slider'
                )
            ], style=slider_style),
            html.Div([
                html.P("Affordability", id='affordability-label', style={'color': '#333333'}),
                dcc.Slider(
                    id='slider-5',
                    min=0,
                    max=5,
                    step=1,
                    value=1,
                    marks={i: str(i) for i in range(6)},
                    className='blue-slider'
                )
            ], style=slider_style)
        ], style={'width': '20%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '20px', 'backgroundColor': '#f7f7f7', 'borderLeft': '1px solid #ccc'})
    ], style={'display': 'flex', 'flexDirection': 'row', 'width': '100%'}),
    
    html.Div([
        html.H3("City Scores Table", id='city-scores-title', style={'textAlign': 'center', 'color': '#333333'}),
        html.Table(id='city-scores-table', style={'width': '100%', 'textAlign': 'center', 'color': '#333333', 'backgroundColor': '#ffffff', 'border': '1px solid #ccc', 'borderRadius': '5px'})
    ], style={'width': '100%', 'display': 'inline-block', 'padding': '20px', 'backgroundColor': '#ffffff', 'boxSizing': 'border-box'})
], style={'backgroundColor': '#f0f0f0', 'fontFamily': 'Arial, sans-serif', 'boxSizing': 'border-box'})
@app.callback(
    Output('map', 'figure'),
    [Input('slider-1', 'value'),
     Input('slider-2', 'value'),
     Input('slider-3', 'value'),
     Input('slider-4', 'value'),
     Input('slider-5', 'value')]
)
def update_map(weight1, weight2, weight3, weight4, weight5):
    # Calculate weighted average
    df['WeightedAvg'] = (weight1 * df['Environment'] + 
                         weight2 * df['Society'] + 
                         weight3 * df['Access'] + 
                         weight4 * df['Prospect'] + 
                         weight5 * df['Affordability'])

    # Create the map
    fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", size="WeightedAvg", 
                            hover_name="City", 
                            mapbox_style="carto-positron", 
                            size_max=15, zoom=3,
                            color_discrete_sequence=px.colors.qualitative.Dark2)
    
    return fig

@app.callback(
    Output('city-value', 'children'),
    [Input('city-dropdown', 'value')]
)
def update_city_value(selected_city):
    city_data = df[df['City'] == selected_city]
    return html.Div([
        html.H4(f"Values for {selected_city}", style={'color': '#333333'}),
        html.P(f"Environment: {city_data['Environment'].values[0]}", style={'color': '#333333'}),
        html.P(f"Society: {city_data['Society'].values[0]}", style={'color': '#333333'}),
        html.P(f"Access: {city_data['Access'].values[0]}", style={'color': '#333333'}),
        html.P(f"Prospect: {city_data['Prospect'].values[0]}", style={'color': '#333333'}),
        html.P(f"Affordability: {city_data['Affordability'].values[0]}", style={'color': '#333333'})
    ])

@app.callback(
    Output('city-dropdown', 'value'),
    Input('map', 'clickData')
)
def update_city_from_map(clickData):
    if clickData is None:
        return dash.no_update
    return clickData['points'][0]['hovertext']

@app.callback(
    Output('city-rank', 'children'),
    [Input('slider-1', 'value'),
     Input('slider-2', 'value'),
     Input('slider-3', 'value'),
     Input('slider-4', 'value'),
     Input('slider-5', 'value')]
)
def update_city_rank(weight1, weight2, weight3, weight4, weight5):
    # Calculate weighted average
    df['WeightedAvg'] = (weight1 * df['Environment'] + 
                         weight2 * df['Society'] + 
                         weight3 * df['Access'] + 
                         weight4 * df['Prospect'] + 
                         weight5 * df['Affordability'])
    
    ranked_cities = df.sort_values(by='WeightedAvg', ascending=False)['City']
    return html.Ul([html.Li(city, style={'color': '#333333'}) for city in ranked_cities])

@app.callback(
    Output('city-scores-table', 'children'),
    [Input('slider-1', 'value'),
     Input('slider-2', 'value'),
     Input('slider-3', 'value'),
     Input('slider-4', 'value'),
     Input('slider-5', 'value')]
)
def update_city_scores_table(weight1, weight2, weight3, weight4, weight5):
    # Calculate weighted average
    df['WeightedAvg'] = (weight1 * df['Environment'] + 
                         weight2 * df['Society'] + 
                         weight3 * df['Access'] + 
                         weight4 * df['Prospect'] + 
                         weight5 * df['Affordability'])

    # Sort cities by weighted average
    sorted_df = df.sort_values(by='WeightedAvg', ascending=False)

    # Create table header
    header = html.Tr([
        html.Th("City"),
        html.Th("Environment"),
        html.Th("Society"),
        html.Th("Access"),
        html.Th("Prospect"),
        html.Th("Affordability"),
        html.Th("Weighted Avg")
    ])

    # Create table rows
    rows = [html.Tr([
        html.Td(city),
        html.Td(sorted_df.loc[sorted_df['City'] == city, 'Environment'].values[0]),
        html.Td(sorted_df.loc[sorted_df['City'] == city, 'Society'].values[0]),
        html.Td(sorted_df.loc[sorted_df['City'] == city, 'Access'].values[0]),
        html.Td(sorted_df.loc[sorted_df['City'] == city, 'Prospect'].values[0]),
        html.Td(sorted_df.loc[sorted_df['City'] == city, 'Affordability'].values[0]),
        html.Td(sorted_df.loc[sorted_df['City'] == city, 'WeightedAvg'].values[0])
    ]) for city in sorted_df['City']]

    # Combine header and rows
    table = [header] + rows

    return table

@app.callback(
    [Output('city-value-title', 'children'),
     Output('city-rank-title', 'children'),
     Output('map-title', 'children'),
     Output('map-subtitle', 'children'),
     Output('scores-title', 'children'),
     Output('environment-label', 'children'),
     Output('society-label', 'children'),
     Output('access-label', 'children'),
     Output('prospects-label', 'children'),
     Output('affordability-label', 'children'),
     Output('city-scores-title', 'children')],
    [Input('language-switch', 'n_clicks')]
)
def update_language(n_clicks):
    if n_clicks % 2 == 1:
        return ["Valeur de la ville", "Classement des villes", "Carte", "Mettre en évidence les villes sélectionnées et le Canada lui-même", 
                "Scores", "Environnement", "Société", "Accès", "Perspectives", "Abordabilité", "Tableau des scores des villes"]
    else:
        return ["City Value", "City Rank", "Map", "Highlight selected cities and Canada itself", 
                "Scores", "Environment", "Society", "Access", "Prospects", "Affordability", "City Scores Table"]

# Add custom CSS for slider styles
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <style>
            .blue-slider .rc-slider-track {
                background-color: #1f77b4 !important;
            }
            .blue-slider .rc-slider-handle {
                border-color: #1f77b4 !important;
            }
            body {
                font-family: Arial, sans-serif;
                background-color: #f0f0f0;
            }
            h1, h2, h3, h4, p {
                margin: 0;
                padding: 0;
            }
            .main-container {
                max-width: 1200px;
                margin: 0 auto;
                padding: 20px;
            }
        </style>
    </head>
    <body>
        <div class="main-container">
            {%app_entry%}
        </div>
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)  # Change port number if 8050 is already in use
